## WeatherPy

#### In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world of varying distance from the equator. To accomplish this, you'll be utilizing a [simple Python library](https://pypi.python.org/pypi/citipy), the [OpenWeatherMap API](https://openweathermap.org/api), and a little common sense to create a representative model of weather across world cities.

#### Your objective is to build a series of scatter plots to showcase the following relationships:

* Temperature (F) vs. Latitude
* Humidity (%) vs. Latitude
* Cloudiness (%) vs. Latitude
* Wind Speed (mph) vs. Latitude

#### Your final notebook must:

* Randomly select **at least** 500 unique (non-repeat) cities based on latitude and longitude.
* Perform a weather check on each of the cities using a series of successive API calls.
* Include a print log of each city as it's being processed with the city number and city name.
* Save both a CSV of all data retrieved and png images for each scatter plot.

#### As final considerations:

* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* You must include a written description of three observable trends based on the data.
* You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
* See [Example Solution](WeatherPy_Example.pdf) for a reference on expected format.

## Hints and Considerations

* The city data is generated based on random coordinates; as such, your outputs will not be an exact match to the provided starter notebook.

* You may want to start this assignment by refreshing yourself on the [geographic coordinate system](http://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm).

* Next, spend the requisite time necessary to study the OpenWeatherMap API. Based on your initial study, you should be able to answer  basic questions about the API: Where do you request the API key? Which Weather API in particular will you need? What URL endpoints does it expect? What JSON structure does it respond with? Before you write a line of code, you should be aiming to have a crystal clear understanding of your intended outcome.

* A starter code for Citipy has been provided. However, if you're craving an extra challenge, push yourself to learn how it works: [citipy Python library](https://pypi.python.org/pypi/citipy). Before you try to incorporate the library into your analysis, start by creating simple test cases outside your main script to confirm that you are using it correctly. Too often, when introduced to a new library, students get bogged down by the most minor of errors -- spending hours investigating their entire code -- when, in fact, a simple and focused test would have shown their basic utilization of the library was wrong from the start. Don't let this be you!

* Part of our expectation in this challenge is that you will use critical thinking skills to understand how and why we're recommending the tools we are. What is Citipy for? Why would you use it in conjunction with the OpenWeatherMap API? How would you do so?

* In building your script, pay attention to the cities you are using in your query pool. Are you getting coverage of the full gamut of latitudes and longitudes? Or are you simply choosing 500 cities concentrated in one region of the world? Even if you were a geographic genius, simply rattling 500 cities based on your human selection would create a biased dataset. Be thinking of how you should counter this. (Hint: Consider the full range of latitudes).

* Lastly, remember -- this is a challenging activity. Push yourself! If you complete this task, then you can safely say that you've gained a strong mastery of the core foundations of data analytics and it will only go better from here. Good luck!

# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Actions in terminal before we begin

# pip install citipy
# add Weather Map api_key to config

# Dependencies and Setup
import openweathermapy.core as owm
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        city = city.title()
        cities.append(city)

# Print the city count to confirm sufficient count
# Do we need to make city names Capitalized?
print(cities)



['Nouadhibou', 'Mataura', 'Kourou', 'Kisangani', 'Atuona', 'Komsomolskiy', 'Ayr', 'Waingapu', 'Pisco', 'Sorvag']


In [4]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="
query_url


'http://api.openweathermap.org/data/2.5/weather?appid=cf61488b0b6c2bf530c649e1dc953891&units=imperial&q='

In [5]:
city = []
lat = []
lon = []
temp_f = []
humidity = []
cloudiness = []
wind_mph = []

for city in cities:
    response = requests.get(query_url + city).json()
    
    if response['cod'] == 200: #checks if response is sucessfull, if other than 200, skip code below
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
        temp_f.append(response['main']['temp'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_mph.append(response['wind']['speed'])

print(cities)
print(lat)
print(lon)
print(temp_f)
print(humidity)
print(cloudiness)
print(wind_mph)

['Nouadhibou', 'Mataura', 'Kourou', 'Kisangani', 'Atuona', 'Komsomolskiy', 'Ayr', 'Waingapu', 'Pisco', 'Sorvag']
[20.93, -46.19, 5.16, 0.52, -9.8, 67.55, 55.46, -9.65, -13.71]
[-17.03, 168.86, -52.64, 25.21, -139.03, 63.78, -4.63, 120.26, -76.2]
[69.8, 64, 87.8, 82.33, 81.52, 26.89, 48.4, 72.25, 71.6]
[56, 69, 62, 81, 100, 90, 53, 100, 30]
[0, 36, 68, 36, 68, 88, 40, 32, 8]
[20.8, 3, 13.87, 1.25, 13, 19.6, 5.82, 2.37, 27.51]


In [6]:
weather_dict = {
    "city": cities,
    "temp": temp_f,
    "humidity": humidity,
    "cloudiness": cloudiness,
    "windspeed": wind_mph,
    "lat": lat,
    "long": lon,
}
weather_data = pd.DataFrame(weather_dict)
weather_data
# weather_dict

# print(f'The number of observations in our dataset is {len(weather_data)}')

# weather_data.head()



ValueError: arrays must all be same length

In [ ]:
settings = {"units": "imperial", "appid": api_key}
current_weather = []

for city in cities:
    current_weather = owm.get_current(city, **settings)

print(f"Current weather object for {city}: {current_weather}.")

In [ ]:

current_weather_paris = owm.get_current({cities}, **settings)
print(f"Current weather object for Paris: {current_weather_paris}.")

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

settings = {"units": "imperial", "appid": api_key}
current_weather_paris = owm.get_current("Paris", **settings)
print(f"Current weather object for Paris: {current_weather_paris}.")



# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="

for city in cities:
    response = requests.get(query_url + city).json()
    #  lat_lngs.append(response['coord']['lat'])
    #  temp.append(response['main']['temp'])

print(response)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot